In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
total_acidity=df["fixed acidity"]+df["volatile acidity"]

In [ ]:
total_acidity

In [ ]:
df.drop(columns=["fixed acidity","volatile acidity"],inplace=True)

In [ ]:
df["Total_Acidity"]=total_acidity
df

In [ ]:
df["quality"].unique()

In [ ]:
df.info()

In [ ]:
for ind in df.index:
    if df["quality"][ind] == 5 :
        print(df.loc[ind])
        print()

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),cmap="YlGnBu",annot=True)

In [ ]:
for i in  df.columns:
    sns.boxplot(df[i])
    plt.xticks(rotation = 90)
    plt.grid()
    plt.show()

In [ ]:
plt.figure(figsize=(10,5))
df.boxplot(rot=55,fontsize=15)
plt.xticks(rotation=90)

In [ ]:
df.describe()

In [ ]:
df["quality"].values

In [ ]:
df["quality"].unique()

In [ ]:
category = []
for num in df["quality"].values :
    if num < 5:
        category.append("Poor")
    elif num > 5:
        category.append("Good")
    else:
        category.append("Mid")

In [ ]:
df

In [ ]:
category = pd.DataFrame(data=category , columns=["category"])
df['category'] = category
df

In [ ]:
sns.barplot(x=df["quality"] , y=df["alcohol"] , palette="Reds")

In [ ]:
sns.barplot(x=df["category"] , y=df["alcohol"] , palette="Reds")

In [ ]:
y = df["category"]
x = df.drop(columns=["quality","category"]) 

In [ ]:
print(x,y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)


In [ ]:
y = y.reshape(-1,1)
print(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0,test_size=0.25)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [ ]:
from sklearn.linear_model import LogisticRegression

lo_re=LogisticRegression()
lo_re.fit(x_train,y_train)
print_score(lo_re,x_train,y_train,x_test,y_test,train=True)
print_score(lo_re,x_train,y_train,x_test,y_test,train=False)

In [ ]:
test_score = accuracy_score(y_test, lo_re.predict(x_test)) * 100
train_score = accuracy_score(y_train, lo_re.predict(x_train)) * 100

results_df = pd.DataFrame(data=[["Logistic Regression", train_score, test_score]], 
                          columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
results_df

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kn_cl = KNeighborsClassifier()
kn_cl.fit(x_train, y_train)

print_score(kn_cl, x_train, y_train, x_test, y_test, train=True)
print_score(kn_cl, x_train, y_train, x_test, y_test, train=False)

In [ ]:
test_score = accuracy_score(y_test, kn_cl.predict(x_test)) * 100
train_score = accuracy_score(y_train, kn_cl.predict(x_train)) * 100


results_df_2 = pd.DataFrame(data=[["K-nearest neighbors", train_score, test_score]], 
                          columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

In [ ]:
from sklearn.svm import SVC

sc_cl = SVC(kernel="rbf")
sc_cl.fit(x_train, y_train)

print_score(sc_cl, x_train, y_train, x_test, y_test, train=True)
print_score(sc_cl, x_train, y_train, x_test, y_test, train=False)

In [ ]:
test_score = accuracy_score(y_test, sc_cl.predict(x_test)) * 100
train_score = accuracy_score(y_train, sc_cl.predict(x_train)) * 100

results_df_2 = pd.DataFrame(data=[["Support Vector Machine", train_score, test_score]], 
                          columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

In [ ]:
from sklearn.tree import DecisionTreeClassifier


tree_clf = DecisionTreeClassifier(random_state=0)
tree_clf.fit(x_train, y_train)

print_score(tree_clf, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf, x_train, y_train, x_test, y_test, train=False)

In [ ]:
test_score = accuracy_score(y_test, tree_clf.predict(x_test)) * 100
train_score = accuracy_score(y_train, tree_clf.predict(x_train)) * 100

results_df_2 = pd.DataFrame(data=[["Decision Tree Classifier", train_score, test_score]], 
                          columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(x_train, y_train)

print_score(tree_clf, x_train, y_train, x_test, y_test, train=True)
print_score(tree_clf, x_train, y_train, x_test, y_test, train=False)

In [ ]:
test_score = accuracy_score(y_test, rf_clf.predict(x_test)) * 100
train_score = accuracy_score(y_train, rf_clf.predict(x_train)) * 100

results_df_2 = pd.DataFrame(data=[["Random Forest Classifier", train_score, test_score]], 
                          columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
results_df = results_df.append(results_df_2, ignore_index=True)
results_df